<a href="https://colab.research.google.com/github/mohmadAyman75/Face-classification/blob/main/only_5_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
upload = files.upload()

Saving Aligned_Faces.zip to Aligned_Faces.zip


In [2]:
import os
import cv2
import numpy as np
import zipfile
import tempfile
ZIP_PATH = "Aligned_Faces.zip"
IMG_SIZE = 224
TARGET_IMAGES = 20
temp_dir = tempfile.mkdtemp()
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)
DATASET_DIR = temp_dir
if os.path.isdir(os.path.join(DATASET_DIR, 'Aligned_Faces')):
    DATASET_DIR = os.path.join(DATASET_DIR, 'Aligned_Faces')
contents = [d for d in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, d))]
if len(contents) == 1 and contents[0] == 'Aligned_Faces':
    DATASET_DIR = os.path.join(DATASET_DIR, 'Aligned_Faces')

print("Dataset extracted to", DATASET_DIR)

Dataset extracted to /tmp/tmpyjbrr0ei/Aligned_Faces


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.15,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True
)

def generate_augmented_images(image, save_dir, start_index, n):
    image = np.expand_dims(image, axis=0)
    gen = datagen.flow(image, batch_size=1)

    for i in range(n):
        aug = next(gen)[0].astype("uint8")
        out_path = os.path.join(save_dir, f"aug_{start_index + i}.jpg")
        cv2.imwrite(out_path, aug)


In [4]:
persons = os.listdir(DATASET_DIR)
persons = persons[:5]

for person in persons:
    pf = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(pf):
        continue

    imgs = [f for f in os.listdir(pf) if f.lower().endswith((".jpg", ".png"))]

    count = len(imgs)
    print(f"[Processing] {person} → {count}")

    if count == 0:
        print(f"No images found for {person}. Skipping augmentation.\n")
        continue

    if count >= TARGET_IMAGES:
        continue

    missing = TARGET_IMAGES - count

    base_img = cv2.imread(os.path.join(pf, imgs[0]))
    base_img = cv2.resize(base_img, (IMG_SIZE, IMG_SIZE))

    generate_augmented_images(base_img, pf, count, missing)
    print(f"Added {missing} images\n")

print("\nAugmentation Done")


[Processing] Alan_Dreher → 1
Added 19 images

[Processing] Barbara_Bach → 1
Added 19 images

[Processing] Ilham_Aliev → 1
Added 19 images

[Processing] Jeane_Kirkpatrick → 1
Added 19 images

[Processing] Stefan_Tafrov → 1
Added 19 images


Augmentation Done


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os

images = []
labels = []
persons = os.listdir(DATASET_DIR)
persons = persons[:5]

for person in persons:
    pf = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(pf):
        continue

    for f in os.listdir(pf):
        if f.lower().endswith((".jpg", ".png", ".jpeg")):
            path = os.path.join(pf, f)
            img = cv2.imread(path)
            if img is None:
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            images.append(img)
            labels.append(person)

images = np.array(images)
labels = np.array(labels)

print("Images:", images.shape)
print("Labels:", labels.shape)


Images: (100, 224, 224, 3)
Labels: (100,)


In [6]:
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(
    images, labels_encoded,
    test_size=0.2,
    stratify=labels_encoded,
    random_state=42
)

print("Train:", X_train.shape)
print("Test:", X_test.shape)


Train: (80, 224, 224, 3)
Test: (20, 224, 224, 3)


In [7]:
X_train = X_train.astype("float32") / 255.0
X_test  = X_test.astype("float32") / 255.0
from tensorflow.keras.utils import to_categorical
num_classes = len(np.unique(labels_encoded))
y_train= to_categorical(y_train, num_classes)
y_test= to_categorical(y_test, num_classes)
print("OneHot Shapes:", y_train.shape, y_test.shape)

OneHot Shapes: (80, 5) (20, 5)
